In [ ]:
pwd

In [ ]:
!pip install keras==2.2.4
!pip install tensorflow==2.6.0

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.layers import MaxPooling2D

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_da259d9bd31c4be3b14760384dcca5e9 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_da259d9bd31c4be3b14760384dcca5e9 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_da259d9bd31c4be3b14760384dcca5e9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZKX_GO0XQV9_8khWZeAYKLc-1jhi6rTfyFa-WeAOk69Z',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_da259d9bd31c4be3b14760384dcca5e9)

streaming_body_1 = client_da259d9bd31c4be3b14760384dcca5e9.get_object(Bucket='icebergdetection-donotdelete-pr-bb7ad9vhmeqyci', Key='Iceberg Detection dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [ ]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [ ]:
pwd

In [ ]:
import os
filenames = os.listdir('/home/wsuser/work/Iceberg Detection dataset/dataset/train')

In [ ]:
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/Iceberg Detection dataset/dataset/train',target_size=(75,75),batch_size= 32, class_mode='binary')
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/Iceberg Detection dataset/dataset/test',target_size=(75,75),batch_size= 32, class_mode='binary')

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(64,(3,3), activation='relu', input_shape=(75,75,3)))

In [ ]:
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(units= 512, kernel_initializer="uniform", activation="relu"))
model.add(Dense(units = 256, kernel_initializer="uniform", activation="relu"))
model.add(Dense(units = 1, kernel_initializer="uniform", activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.fit_generator(x_train,steps_per_epoch=40, epochs= 10, validation_data=x_test, validation_steps=10)

In [ ]:
model.save("iceberg.h5")

In [ ]:
!tar -zcvf image-classification-model_new.tgz iceberg.h5

In [ ]:
ls -1

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"5wyIz3tUi45QyVxwp6msPJ-vusw7ovWgj0iz8P0JOpop"
                  }

client = APIClient(wml_credentials)

In [ ]:
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client, 'iceberg_detection_deploy')
print("Space UID = " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})

model_id = client.repository.get_model_uid(model_details)

In [ ]:
model_id

In [ ]:
client.repository.download(model_id, 'my_models.tar.gz')

In [ ]:
from keras.models import load_model
from keras.preprocessing import image

In [ ]:
model = load_model("iceberg.h5")

In [ ]:
streaming_body_11 = client_da259d9bd31c4be3b14760384dcca5e9.get_object(Bucket='icebergdetection-donotdelete-pr-bb7ad9vhmeqyci', Key='f213.png')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
image = image.load_img(streaming_body_11, target_size = (128,128))

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
x_train.class_indices